### 插入数据库

In [1]:
from llama_index.core import (
    Document,
    VectorStoreIndex,
    Settings,
    SimpleDirectoryReader,
)
from typing import Optional, List, Mapping, Any
import chromadb
from VectorDB import VectorDBManager
import os, re, json
from pprint import pprint

# from transcribe import stream_transcribe
from llm import DashScopeLLM
from llama_index.core.node_parser import TextSplitter


Settings.llm = DashScopeLLM()
DATA_DIR = "/root/autodl-tmp/RAG/docs/chunks1.json"
CHROMA_PERSIST_DIR = "/root/autodl-tmp/RAG/chroma_db"
embed_models = {
    'bge-m3': '/root/autodl-tmp/models/BAAI/bge-m3'    
}
vector_db = VectorDBManager("/root/autodl-tmp/RAG/chroma_db")


当前向量数据库为：/root/autodl-tmp/RAG/chroma_db
数据库包含集合: ['templ1']


基于txt文档的检索

In [ ]:
def read_and_clean_txt(file_path: str) -> str:
    with open(file_path, 'r', encoding='gbk') as f:
        lines = f.readlines()
    # 去除每行首尾空白，并过滤掉空行，然后去掉每行所有空白符（包括空格、制表符等）
    cleaned_lines = [re.sub(r'\s+', '', line) for line in lines if line.strip()]
    return '\n'.join(cleaned_lines)

def split_by_disease(text: str) -> List[dict]:
    # 存储切分后的结果
    disease_chunks = []
    current_disease = None
    current_content = []
    
    # 按行处理文本
    lines = text.splitlines()
    
    for line in lines:
        # 跳过空行
        if re.search(r'^(\[\d+\])|(\[\d+\]$)', line):
            continue
            
        # 识别疾病标题 - 通过【】或者数字.作为标志
        if line.startswith('【'):
            # 保存上一个疾病的内容
            if current_disease and current_content:
                disease_chunks.append({
                    'disease_name': current_disease,
                    'content': '\n'.join(current_content)
                })
            
            # 提取新的疾病名称
            if line.startswith('【'):
                current_disease = line.strip('【】')
            else:
                current_disease = line.split('.', 1)[1].strip()
            
            current_content = [line]
        else:
            if current_disease:
                current_content.append(line)
    
    # 保存最后一个疾病的内容
    if current_disease and current_content:
        disease_chunks.append({
            'disease_name': current_disease,
            'content': '\n'.join(current_content)
        })
    
    return disease_chunks

# 示例用法
txt_file_path = '/root/autodl-tmp/RAG/SpeechAutoReporter/templates/doc1.txt'
cleaned_text = read_and_clean_txt(txt_file_path)

class DiseaseNodeParser(TextSplitter):
    def split_text(self, text: str) -> List[str]:
        # 返回字符串列表而不是Document对象
        disease_chunks = split_by_disease(text)
        return [chunk['content'] for chunk in disease_chunks]

node_parser = DiseaseNodeParser()
sp = split_by_disease(cleaned_text)
pprint(sp)
vector_db.insert_or_update_documents('templ1',[cleaned_text],['doc1'],node_parser=node_parser)
vector_db.view_all_documents()

大模型病例输出测试

In [2]:
def process_audio_content(audio_content: str) -> dict:
    
    prompt = """请根据以下医疗问诊语音识别的对话内容，识别说话人身份（医生/患者），由于原始内容来自语音识别可能存在字词错误，请你进行修正,可以对错误字词替换，增添或者删除。
                输出格式要求：{"content"：[{"身份":"内容"},{"身份":"内容"}], "subject":"总结内容"}
                1. 身份只能来自：医生、患者、家属、噪声四类。
                2. 内容是说话人说的话
                3. 总结内容是对整个对话的总结，要求简洁明了，包含主要信息(来自什么科室的对话，疾病部位，症状, 可能的疾病)，尽可能简洁准确便于后续和模板匹配
                格式化答案部分用###包裹，便于后续处理，包裹部分内容确保为可以被jsons.loads()解析的格式化字符串
                如
                ###
                {"content":[{"医生":"你好，我是医生，请问有什么问题？"},{"患者":"我最近感觉头痛，想咨询一下。"}],
                "subject":"患者头痛，咨询医生"}
                ###
                注意，修正不正确的字词非常重要，因为其是后续基于文本检索的基础参考，请多考虑多音字、名称相关性等可能进行正确词汇猜测，保证修正句子不存在意义不明字词，前后逻辑顺畅没有冲突。
                对于总结内容，需要对病人异常症状进行总结概括，避免出现对话中没有的内容和推测的结果。
                原始内容：
            """ + f'{audio_content}'
    
    # 调用LLM处理
    response = Settings.llm.complete(prompt)
    try:
        # 解析LLM返回的JSON格式内容
        pattern = r'###(.*?)###'
        match = re.search(pattern, response.text, re.DOTALL)
        if match:
            res = match.group(1).strip()
        result = json.loads(res)
        print(f"处理结果：{result}")
        return result
    except json.JSONDecodeError:
        print("LLM返回格式解析失败，返回原始格式")

def generate_medical_description(retrieved_contents: List, dialog_content: List[dict]) -> str:
    # 构建上下文：将检索到的相关模板组合
    templates = "\n".join([f"模板{i+1}：{contents}" for i, contents in enumerate(retrieved_contents)])
    
    # 构建对话内容
    # Convert dialog content list to formatted string
    dialog = json.dumps(dialog_content, ensure_ascii=False)
    
    prompt = f"""基于检索到的超声报告模板和患者对话内容，生成规范的病例描述。
        参考模板内容：
        {templates}

        患者对话记录：
        {dialog}

        要求：
        1. 严格参考模板的术语和表达方式
        2. 保持医学描述的专业性和规范性
        3. 如果检索到的多个模板有重复或冗余内容，需要在最相关目标模板中进行整合
        4. 输出内容应该是根据模板条目纠正过的规范化病例描述，而不是单纯复制报告模板
        5. 最重要的是，病例报告仅能出现对话部分诊断，其他额外的内容和推测的结果可能导致严重医疗事故，因此严禁出现对话内容外的表述和推出。
        请生成规范的病例描述：
    """

    response = Settings.llm.complete(prompt)
    print(f"生成的病例描述：{response.text}")
    return 

sample = '''
    子宫增大，多个肌壁间低回声，边界不清，周边环状血流。造影增强30秒起，早期环形高增强，网状填充，峰值不均，晚期低增强。提示：多发肌瘤。
'''

res = process_audio_content(sample)
query = res['subject']
content = res['content']
results = vector_db.retrieve('templ1', query, top_k=3)
pprint(results)
contents_list = [item['content'] for item in results if 'content' in item]
generate_medical_description(contents_list, content)

{'choices': [{'message': {'content': '###  \n{\n    "content": [\n        {\n            "医生": "检查结果显示子宫有多发肌壁间低回声，边界不清，周边有环状血流。造影显示早期环形高增强，网状填充，峰值不均匀，晚期低增强。提示可能是多发肌瘤。"\n        }\n    ],\n    "subject": "子宫多发肌瘤诊断"\n}\n###', 'reasoning_content': '好的，我现在需要处理用户的医疗问诊语音识别对话内容。首先，我要仔细阅读原始内容，识别说话人身份。根据内容来看，这似乎是关于子宫的检查结果，提到多个肌壁间低回声、边界不清、环状血流等。造影增强显示早期环形高增强，网状填充，峰值不均，晚期低增强。提示是多发肌瘤。\n\n首先，分析说话人身份。内容中提到子宫增大和多发肌瘤，这通常是医生在讨论患者的检查结果。因此，说话人应该是医生。没有提到患者或家属的直接对话，所以排除患者和家属的可能性。\n\n接下来，检查是否有字词错误。原始内容中的“造影增强30秒起”可能需要调整，因为造影增强通常是指增强期，而不是开始时间。正确的表述应该是“造影增强后，显示早期环形高增强，网状填充，峰值不均，晚期低增强。”\n\n然后，总结内容。主要信息包括子宫多发肌瘤的诊断，造影显示的情况，以及建议进一步检查如超声和MRI。因此，总结内容应该是患者患有子宫多发肌瘤，建议进一步检查。\n\n最后，按照要求，输出格式化为JSON，确保内容准确无误，没有错误字词。确认后，生成正确的JSON格式。', 'role': 'assistant'}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None}], 'object': 'chat.completion', 'usage': {'prompt_tokens': 374, 'completion_tokens': 367, 'total_tokens': 741}, 'created': 1746516605, 'system_fingerprint': None, 'model': 'deepseek-r1-distill-qwen-7b', 'id': 'chatcm